In [152]:
#Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm

from random import random
import math

import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing


In [153]:
df = pd.read_csv("/Users/normalazo/Desktop/env1/Entrega6_CrossValidation/regLinPoli2.csv")

In [154]:
# This data does not have headers so each attribute or field is simply enumerated
df.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X30,X31,X32,X33,X34,X35,X36,X37,X38,y
count,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.00000,1029.000000,1029.000000,1029.000000,...,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1029.000000,1.029000e+03
mean,-12.492680,327718.372035,2.565422,-0.018122,1.235180,-153.782162,-23648.95337,-2.565422,0.018122,-4.108330,...,5.612245,100.650146,-29.719145,-136.164237,-101.649174,-34.310010,-7.965986,-37.117590,2.499514,-8.035132e+18
std,572.608894,298395.344783,0.424105,0.715733,58.709228,0.000000,0.00000,0.424105,0.715733,584.648570,...,239.009325,2862.331915,4489.803784,2641.944700,5153.481524,2569.220416,2933.878621,1504.903744,102.994972,2.593868e+20
min,-992.795572,1.098476,0.020395,-0.999991,-100.000000,-153.782162,-23648.95337,-2.999260,-0.999992,-996.652132,...,-410.000000,-4857.000000,-7715.000000,-4596.000000,-8898.000000,-4383.000000,-5133.000000,-2565.000000,-175.000000,-9.506460e+20
25%,-499.000060,59791.243170,2.388319,-0.760277,-51.000000,-153.782162,-23648.95337,-2.869121,-0.714801,-511.919891,...,-189.000000,-2457.000000,-4063.000000,-2438.000000,-4780.000000,-2311.000000,-2520.000000,-1411.000000,-85.000000,-7.703790e+18
50%,-27.174576,241226.533900,2.691213,-0.029038,3.000000,-153.782162,-23648.95337,-2.691213,0.029038,-6.507760,...,7.000000,123.000000,129.000000,-196.000000,-13.000000,20.000000,-35.000000,78.000000,6.000000,-1.094503e+10
75%,477.216100,547320.664300,2.869121,0.714801,52.000000,-153.782162,-23648.95337,-2.388319,0.760277,516.046297,...,214.000000,2654.000000,3890.000000,2138.000000,4195.000000,2165.000000,2521.000000,1227.000000,94.000000,5.636450e+18
max,998.297367,996597.632200,2.999260,0.999992,100.000000,-153.782162,-23648.95337,-0.020395,0.999991,1000.933758,...,410.000000,4862.000000,7752.000000,4577.000000,8866.000000,4384.000000,5123.000000,2568.000000,175.000000,9.881420e+20


In [155]:
#Dividir los datos en train y test
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[[df.columns[-1]]], train_size=0.75)

In [156]:
# I recommend that after manipulating data using pandas and before modelling to convert dataframes into arrays. 
# This may avoid some headaches
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
Y_train=np.asarray(Y_train)
Y_test=np.asarray(Y_test)

In [157]:
# This procedure is useful for classroom examples. For a real implementation you have to have a separete method 
# for transforming the production data so you can transform it as you get it with the fitted scaler
## The procedure returns a standardized copy of the input data
def normalize(X_train,X_test,Y_train,Y_test,do=True):

    scale_X=preprocessing.StandardScaler()
    scale_y=preprocessing.StandardScaler()
    
    train_X=np.copy(X_train)
    train_y=np.copy(Y_train)
    test_X=np.copy(X_test)
    test_y=np.copy(Y_test)
    if do:
        scale_X.fit(train_X)
        scale_y.fit(train_y)
        train_X=scale_X.transform(train_X)
        train_y=scale_y.transform(train_y)
        test_X=scale_X.transform(test_X)
        test_y=scale_y.transform(test_y)
    return train_X,test_X, train_y, test_y

In [158]:
def ILR(X_train, Y_train, learningC, w, lam ):
    numrows = len(X_train)    
    numcols = len(X_train[0]) 

    for j in range (0,numrows):

        x = np.ones(1)
        x = np.concatenate((x,X_train[j]), axis=0)

        #calculate the value of 
        res = np.dot(w,x)

        #difference 
        dif = Y_train[j] - res

        #print dif

        for i in range (0,numcols +1 ):
            if(i==0):
                w[i] = w[i]+dif*learningC*x[i]
            else:
                w[i] = w[i]+dif*learningC*x[i]-w[i]*lam
            
    return w


In [159]:
def ILR_batch(X_train, Y_train, learningC, w, lam , numBatch, batchSize):
    numrows = len(X_train)    
    numcols = len(X_train[0]) 
    x_barra = np.zeros(numcols+1)
    dif_acum = 0
    

    for j in range (0,numBatch):
        
        idx = np.random.randint(X_train.shape[0], size=batchSize)
        idx
        X_Batch = X_train[idx,:]
        
        
        for k in range (0,batchSize):
            x = np.ones(1)
            x = np.concatenate((x,X_train[k]), axis=0)
            x_barra = x_barra + x
            
            #calculate the value of 
            res = np.dot(w,x)
            
            #difference 
            dif = Y_train[k] - res
            dif_acum = dif_acum + dif
        
        x_barra=x_barra/batchSize
        dif_acum=dif_acum/batchSize

        #actualizamos cada peso
        for i in range (0,numcols +1 ):
            if(i==0):
                w[i] = w[i]+dif_acum*learningC*x_barra[i]
            else:
                w[i] = w[i]+dif_acum*learningC*x_barra[i]-w[i]*lam

    return w


In [160]:
#Numero de Columnas y Renglones
numrows = len(X_train)    # 3 rows in your example
numcols = len(X_train[0]) # 2 columns in your example

In [161]:
#Dividir en k subsets

#inicializacion de vector de w's

# returns a matrix with k subsets.

def k_batch(k, X_train):
    numrows = len(X_train)
    numcols = len(X_train[0]) 
    
    w = np.random.randint(k, size=numrows)
    
    #Tamaño maximo del subset dos veces el numero de rows entre k
    size = (numrows/k)*2
    matrix = np.zeros((k,size,numcols))
    
    for s in range (0,k):
        for j in range (0,size):
            matrix[s][j][0] = -9999

    for i in range (0,k):
        count = 0
        for j in range (0, numrows):
            if(w[j]==i):
                matrix[i][count] = X_train[j]
                count = count + 1
        count = 0
    

    #Guardar el número de elementos que hay en cada subset en un arreglo de tamaño k
    i=0
    elements = 0
    size = (numrows/k)*2
    numElements = np.zeros(10)

    for w in range (0,10):
        elements = elements + i
        i = 0
        while( matrix[w][i][0] <> -9999 and i<=100 ):
            i = i +1
        numElements[w] = i
    
    #juntar en un arreglo k-1 subsets 
    rand = np.random.randint(10)
    numElementsL = numrows - numElements[rand]
    numElementsL = int(numElementsL)
    k_1 = np.zeros((numElementsL,numcols))
    
    numElemento = 0

    for i in range (0,10):
        if(i <> rand):
            for j in range(0,int(numElements[i])):
                k_1[numElemento] = matrix[i][j]
                numElemento = numElemento + 1
                
    #Separar el test subset
    numElementsTest = int(numElements[rand])
    k_Test = np.zeros((numElementsTest,numcols))
    
    for i in range(0,numElementsTest):
        k_Test[i] = matrix[rand][i]

    return matrix, k_1, k_Test
        


# Implementacion Cross Validation

In [162]:
# Dividir los datos en train y test.
train, test = train_test_split(df, train_size=0.75)

# Convertir en arreglos
train=np.asarray(train)
test=np.asarray(test)


Separar en k subsets.

Escoger aleatoriamente 1 para test y k-1 para entrenamiento.

La funcion k_batch(k, dataset) Recibe dos parametros:
    k: El número de subsets en los que queremos dividir nuestro set de datos.
    dataset: El set de datos se entrenamiento para hacer cross validation. 

Nos regresa tres arreglos. 
1. Arreglo que contiene los k subsets.
2. Arreglo que contiene los k-1 subsets que se van usar para entrenamiento.
3. Arreglo con el subset que se va a usar para validacion de resultados.

In [163]:
matrix_a, ksubset, ktest = k_batch(10,train)

ksubset = pd.DataFrame(data=ksubset[0:,0:])
ktest = pd.DataFrame(data=ktest[0:,0:])

#Separa Columnas X and Y
X_ksubset = ksubset[ksubset.columns[0:-1]]
X_ktest = ktest[ktest.columns[0:-1]]

#print X_ktest

Y_ksubset = ksubset[ksubset.columns[-1]]
Y_ktest = ktest[ktest.columns[-1]]


# Transformar en arreglos para manejarlo más facil.
X_ksubset=np.asarray(X_ksubset)
X_ktest=np.asarray(X_ktest)
Y_ksubset=np.asarray(Y_ksubset)
Y_ktest=np.asarray(Y_ktest)

#means = np.mean(ksubset, axis=0)
#stds = np.std(ksubset, axis=0)

Estandarizo mis subsets de acuerdo a los valores de 'ksubset' que es el set de datos de entrenamiento.

In [164]:
Y_ksubset = Y_ksubset.reshape(-1,1)
Y_ktest = Y_ktest.reshape(-1,1)

scalerX = preprocessing.StandardScaler()
scalerY = preprocessing.StandardScaler()

scalerX.fit(X_ksubset)
scalerY.fit(Y_ksubset)

X_ksubset=scalerX.transform(X_ksubset)
Y_ksubset=scalerY.transform(Y_ksubset)
X_ktest=scalerX.transform(X_ktest)
Y_ktest=scalerY.transform(Y_ktest)

Tenemos que hacer pruebas para diferentes valores de lambda y verificar cual parametro es el que me genera un menor error.

In [165]:
# ILR(X_train, Y_train, learningC, w, lam ):

lam1 = .01
lam2 = .02
lam3 =  .03
lam4 =  .025

#lam1
w = np.ones(numcols+1)
for j in range (0,10):
    w = ILR(X_ksubset, Y_ksubset, lam1, w, 0.0)
    X_mod = np.c_[ X_ktest, np.ones(X_ktest.shape[0]) ]
    Y_pred = np.dot(w,X_mod.T)

print "CASO 1"
print "Lambda = ", lam1
print "MSE = " , mean_squared_error(Y_ktest, Y_pred) , "\n"

#lam2
w = np.ones(numcols+1)
for j in range (0,10):
    w = ILR(X_ksubset, Y_ksubset, lam2, w, 0.0)
    X_mod = np.c_[ X_ktest, np.ones(X_ktest.shape[0]) ]
    Y_pred = np.dot(w,X_mod.T)

print "CASO 2"
print "Lambda = ", lam2
print "MSE = " , mean_squared_error(Y_ktest, Y_pred) , "\n"

#lam3
w = np.ones(numcols+1)
for j in range (0,10):
    w = ILR(X_ksubset, Y_ksubset, lam3, w, 0.0)
    X_mod = np.c_[ X_ktest, np.ones(X_ktest.shape[0]) ]
    Y_pred = np.dot(w,X_mod.T)

print "CASO 3"
print "Lambda = ", lam3
print "MSE = " , mean_squared_error(Y_ktest, Y_pred) , "\n"

#lam4
w = np.ones(numcols+1)
for j in range (0,10):
    w = ILR(X_ksubset, Y_ksubset, lam4, w, 0.0)
    X_mod = np.c_[ X_ktest, np.ones(X_ktest.shape[0]) ]
    Y_pred = np.dot(w,X_mod.T)

print "CASO 4"
print "Lambda = ", lam4
print "MSE = " , mean_squared_error(Y_ktest, Y_pred) , "\n"



CASO 1
Lambda =  0.01
MSE =  3.69188830467 

CASO 2
Lambda =  0.02
MSE =  3.89523213289 

CASO 3
Lambda =  0.03
MSE =  4.46480083584 

CASO 4
Lambda =  0.025
MSE =  4.12108641935 

